# HTS analysis 

This notebook contains a prelimiary analysis of the BRT users in the 2019 Household travel survey 2019. 
The focus of this analysis focuses on: 
- % of users living within the catchment areas
- Mode share of the access/eggress leg in a BRT trip
- % of transactions that are transfers. 

In [2]:
import pandas as pd 
import geopandas as gpd 
import numpy as np 
import time 
import matplotlib.pyplot as plt 
import seaborn as sns

In [86]:
output_path = '../data/output/'
input_path = '../data/input/'

catchment_areas = gpd.read_file(output_path + 'tables/catchment_areas/catchment_areas.shp', crs = 'EPSG:4686')\
                     .rename(columns= {'station_na':'station_name'})

#Bogota shapefiles 
lsoas = gpd.read_file(input_path + 'localidades_shp/Loca.shp')
lsoas.drop(index = 8, inplace= True )
lsoas['bog'] = 1
bogota = lsoas.dissolve('bog')



In [6]:
path_to_files = '../../../../02_Data/02_Encuestas de movilidad/03_2019/data_base/csv_files/'
legs_path = 'EtapasEODH2019.csv'
trips_path = 'ViajesEODH2019.csv'
persons_path = 'PersonasEODH2019.csv'
household_path = 'HogaresEODH2019.csv'
vehicle_path = 'Veh°culosEODH2019.csv'

households = pd.read_csv(path_to_files + household_path ,sep=',', dtype = {'Id_Hogar': np.str})#, index_col = 'Id_Hogar')
persons = pd.read_csv(path_to_files + persons_path ,sep=',', dtype = {'id_hogar': np.str})
trips = pd.read_csv(path_to_files + trips_path ,sep=';', dtype = {'id_hogar': np.str})
legs = pd.read_csv(path_to_files + legs_path ,sep=';', dtype = {'id_hogar': np.str})

In [9]:
print('Household sample size:', households.Id_Hogar.nunique())
print ('Persons sample size:', persons.shape[0])
print ('Number of trips:', trips.shape[0])
print ('Number of legs:', legs.shape[0])

Household sample size: 21828
Persons sample size: 66820
Number of trips: 134497
Number of legs: 152310


# Assing new IDs
Household ID is a combination of letter and numbers. Not both types in every ID. I will make a uniform type (int) to every household ID. 

I will also create a unique ID for a person, trip and leg. The current dataset only includes an ID within a household, but not wrt the whole sample size. 

In [10]:
%%time 
#Create new household numerical (with 6 digits) id. 
new_ids = range(100000, 100000 + households.Id_Hogar.nunique())
new_id_df = pd.DataFrame({'Id_Hogar':households.Id_Hogar, 
                          'hh_id': new_ids}).set_index('Id_Hogar')

households = pd.merge(households, new_id_df, how = 'left', left_on = 'Id_Hogar', right_index = True)
persons = pd.merge(persons, new_id_df, how = 'left', left_on = 'id_hogar', right_index = True)
trips = pd.merge(trips, new_id_df, how = 'left', left_on = 'id_hogar', right_index = True)
legs = pd.merge(legs, new_id_df, how = 'left', left_on = 'id_hogar', right_index = True)

CPU times: user 138 ms, sys: 60.7 ms, total: 199 ms
Wall time: 201 ms


In [ ]:
#Asserting correct merging and no missing data
tables = [households, persons, trips, legs]
for table in tables:
    assert table.hh_id.min() == 100000 and table.hh_id.max() == 121827
    assert table.hh_id.isnull().sum() == 0 

In [11]:
#Creating a unique person ID - The two next digits 
persons['person_id'] = (persons.hh_id.astype(str) + persons.id_persona.astype(str).str.zfill(2)).astype(int)
trips['person_id'] = (trips.hh_id.astype(str) + trips.id_persona.astype(str).str.zfill(2)).astype(int)
legs['person_id'] = (legs.hh_id.astype(str) + legs.id_persona.astype(str).str.zfill(2)).astype(int)

#Creating a unique trip ID - The next two digits 
trips['trip_id'] = (trips.person_id.astype(str) + trips.id_viaje.astype(str).str.zfill(2)).astype(int)
legs['trip_id'] = (legs.person_id.astype(str) + legs.id_viaje.astype(str).str.zfill(2)).astype(int)

#Creating a unique leg ID - Last digit 
legs['leg_id'] = (legs.trip_id.astype(str) + legs.id_etapa.astype(str).str.zfill(1)).astype(int)

## Percentage of users living within the catchment areas

In [118]:
# Households IDs that has at least one trip in TM 
BRT_modes = ['TransMilenio', 'Alimentador','Cable']
BRT_hh_id = trips[trips.modo_principal.isin(BRT_modes)].hh_id

# Filter HHs with at least one BRT mode
cols = ['hh_id','Latitud', 'Longitud','Factor', 'municipio', 'localidad','p5_estrato']
BRT_hhs = households[households.hh_id.isin(BRT_hh_id)][cols]

# Transforming Households to a Geopandas object 
BRT_hhs = gpd.GeoDataFrame(BRT_hhs, 
                           geometry = gpd.points_from_xy(BRT_hhs.Longitud, BRT_hhs.Latitud), 
                           crs = 'EPSG:4686')
brt_hhs = len(BRT_hhs)

# Points within the Bogota area
BRT_hhs = gpd.sjoin(trhh_gdf, bogota, op = 'within')
brt_hhs_bogota = len(BRT_hhs)

cols.append('geometry')
print('Households that have at least one trip in TM in Bogota:', brt_hhs_bogota)

#Pople within the catchment area:
ca_people = gpd.sjoin(BRT_hhs[cols], catchment_areas, op = 'within') #catchment_areas.plot()
print('Percetange of people living within the catchment areas:', 100*(len(ca_people))/brt_hhs_bogota)

Households that have at least one trip in TM in Bogota: 6648
Percetange of people living within the catchment areas: 89.62093862815884


TO DO:
- It could be that the BRT did not initiate at home. Filter BRT trips that have origin/destination is home

## BRT Access/egress mode share

In [512]:
def unique_mode(modes_in_list):
    values = []
    for ele in modes_in_list:

        if len(ele)>=2:
            value = ele.min()
        elif len(ele)==1:
            value = ele[0]
        else: 
            value = 0
        values.append(value)
    return values

In [513]:
def shares(df, access = 'access'):
    s = pd.Series(df.groupby(access).weight.sum()).reset_index()
    s['Share(%)'] = (s.weight/s.weight.sum())*100
    s = s.sort_values('Share(%)', ascending = False)
    s = s.set_index(s[access])
    s['Share(%)'] = s['Share(%)'].round(1)
    return s[['Share(%)']]

In [514]:
# Select trips in BRT 
BRT_modes = ['TransMilenio','Cable']
BRT_trips = trips[(trips.modo_principal.isin(BRT_modes))]

BRT_legs = legs[legs.trip_id.isin(BRT_trips.trip_id)]

#Cable, feeder, dual and intercity can be separeted from smartcard data.
#Therefore, we analyses access and eggress from all these. 
BRT_legs['p18_id_medio_transporte'].replace({24:1, 3:1, 2:1, 8:1}, inplace = True)

#Obtain access/egress modes for all TM trips
BRT_legs1 = BRT_legs.groupby('trip_id').agg({'p18_id_medio_transporte': 'unique'})
BRT_legs1 = BRT_legs1.merge(trips[['trip_id', 'f_exp']], left_index = True, right_on = 'trip_id').reset_index()

#Split access and eggress
list_ = []
for row in BRT_legs1['p18_id_medio_transporte']:
    index = np.where(row == 1)[0][0]
    access = pd.Series(np.split(row, [index, index+1]))
    list_.append(access)

access_modes = pd.concat(list_, axis = 1).T
access_modes.rename(columns = {0:'access', 1:'BRT', 2:'egress'}, inplace = True)

#Transforming list to number (easier for manipulation)
access_modes['access'] = unique_mode(access_modes.access)
access_modes['egress'] = unique_mode(access_modes.egress)
access_modes['BRT'] = unique_mode(access_modes.BRT)

# If access or egress = 0, means access/egress = Walk
access_modes.access.where(access_modes.access != 0, 1, inplace = True)
access_modes.egress.where(access_modes.egress != 0, 1, inplace = True)
access_modes

# Transforming number to mode name
dict_replace = {1: 'Walk', 
                4: 'SITP', 5: 'SITP', 6: 'SITP', 7: 'SITP', 
                19: 'Informal',  15: 'Informal', 23: 'Informal', 20: 'Informal', #Biketaxi is informal
                16: 'Taxi', 17: 'Taxi', 18: 'Taxi', 14: 'Taxi',
                35: 'Private vehicle', 36: 'Private vehicle', 32:'Private vehicle', 33:'Private vehicle',
                39: 'Private vehicle',34: 'Private vehicle',
                30: 'Bike - other nm', 31: 'Bike - other nm', 36: 'Bike - other nm', 37: 'Bike - other nm',
                38: 'Bike - other nm', 13: 'Bike - other nm',
                12: 'other',9:'other',10:'other',11:'other',21:'other',22:'other',25:'other', 99: 'other', 
                40: 'Walk'}

access_modes.access.replace(dict_replace, inplace = True)
access_modes.egress.replace(dict_replace, inplace = True)

In [517]:
access_modes['weight'] = BRT_legs1['f_exp']

access_shares = shares(access_modes, access = 'access')
egress_shares = shares(access_modes, access = 'egress')
access_shares.merge(egress_shares, left_index = True, 
                    right_index = True, 
                    suffixes= ['_access', '_egress'])

,Share(%)_access,Share(%)_egress
Walk,88.1,87.5
SITP,6.1,5.9
Informal,3.5,3.8
Taxi,0.8,1.5
Private vehicle,0.7,0.4
other,0.6,0.6
Bike - other nm,0.3,0.3


## Percentage of home transactions that are transfers 

TO DO:
- Get HW location 
- Get Transactions Frequent users 
- Identify HOME transactions 
- From home transactions, create dummy (1. Transfer, 0. No Transfer) 
- Get percentages of transfers and no transfers. 

In [1]:
import pyspark as ps
from pyspark.sql.functions import *
sc = ps.SparkContext(appName="transfers_validations")

In [2]:
from os import path
import time 
import random 
import pandas as pd 
import numpy as np 
import warnings
import glob

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import geopandas as gpd
warnings.filterwarnings('ignore')

from pyspark.sql.functions import *
from pyspark.sql import * #This enables the SparkSession object
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import pow, col, sqrt

spark = SparkSession.builder\
        .master("local") \
        .appName("transfers_validations") \
        .getOrCreate()

In [9]:
output_path = '../data/output/'
input_path = '../data/input/'

#HW locations
hw_location = spark.read.csv(output_path + 'tables/hw_location.csv', header =True, sep = ',')

#Transactions 
transactions = spark.read.csv(output_path + 'tables/transactions_frequent_users.csv',
                              header =True, sep = ',')

In [10]:
transactions

DataFrame[card_id: string, dt: string, fechatransaccion: string, horatransaccion: string, station_name: string, valor: string, diff_seconds: string, transactions: string, active_days: string, max_day: string, min_day: string, time_spam: string, trans_per_timespam: string, trans_per_activeday: string]

In [12]:
hw_location

DataFrame[card_id: string, home_station: string, transacciones_h: string, time_h: string, work_station: string, transacciones_w: string, time_w: string]

In [14]:
# Merge hw_locations with transactions of frequent users 
df = transactions.join(hw_location, on = 'card_id', how = 'outer')

# Identify home transactions 
home_origin = (df['station_name'] == df['home_station']).cast('integer')
df = df.withColumn('home_transaction', home_origin)
home_transactions = df.filter(col('home_transaction') == 1)#.select(col('station_name'), col('time'))

In [15]:
home_transactions.show()

+--------+--------------------+----------------+---------------+--------------------+-----+------------+------------+-----------+----------+----------+---------+------------------+-------------------+--------------------+---------------+-----------------+---------------+---------------+-----------------+----------------+
| card_id|                  dt|fechatransaccion|horatransaccion|        station_name|valor|diff_seconds|transactions|active_days|   max_day|   min_day|time_spam|trans_per_timespam|trans_per_activeday|        home_station|transacciones_h|           time_h|   work_station|transacciones_w|           time_w|home_transaction|
+--------+--------------------+----------------+---------------+--------------------+-----+------------+------------+-----------+----------+----------+---------+------------------+-------------------+--------------------+---------------+-----------------+---------------+---------------+-----------------+----------------+
|10000172|2020-01-02T06:53:...|

In [19]:
home_transactions.select('valor').distinct().collect()

[Row(valor='200'),
 Row(valor='2500'),
 Row(valor='2160'),
 Row(valor='2400'),
 Row(valor='1800')]

In [20]:
home_transactions.groupby('valor').count().show()

+-----+--------+
|valor|   count|
+-----+--------+
|  200| 5113196|
| 2500|25376584|
| 2160| 2474178|
| 2400|28183114|
| 1800| 4994646|
+-----+--------+

